# 01 Selenium 引入概述
思想来源：有时网页内容经过复杂的加密，如果要解密十分麻烦和痛苦。但是既然在浏览器上显示正常，我们能不能让浏览器来完成这些复杂的操作，只接受最后的结果呢？

答案是：有。我们可以利用工具 *selenium*完成这个工作。*selenium*原本并不是一个爬虫工具，而是一个自动化的测试工具，它的功能是打开浏览器，然后像人一样操作浏览器。程序员可以从*selenium*中取得网页中任何想要的东西。

### 环境搭建
- pip install selenium -i 清华源
- 下载浏览器驱动：https://npm.taobao.org/mirrors/chromedriver
- 下载完毕后的exe文件放到python解释器的路径下。

In [ ]:
! pip install selenium

In [2]:
from selenium.webdriver import Chrome

In [7]:
#创建浏览器对象
web=Chrome()
web.get('https://www.baidu.com')
#获取页面标题
print(web.title)


百度一下，你就知道


### selenium 抓清华源
selenium能够模拟所有人类在网页的行为，下面进行展示。

In [14]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
browser = Chrome()
browser.get('https://www.bilibili.com/')#打开B站
el=browser.find_element(By.XPATH,'//*[@id="i_cecream"]/div[2]/div[1]/div[3]/div[2]/div[1]/a[9]')
#找到鬼畜按钮(find_elements_by_xpath等方法现已废除)
el.click()#点击按钮
time.sleep(1)#这里的按钮是一个a标签，如果是ajax重新渲染网页的话，你就不得不刻意等待渲染的完成，否则会报错。
el=browser.find_element(By.XPATH,'//*[@id="nav-searchform"]/div[1]/input').send_keys('python',Keys.ENTER)
#这里的输入框应该是选择输入文字的最内层框。在搜索栏输入'python'

运行上面这个代码，可以发现，完成这样简单的功能需要的时间非常的长。部分是因为下一个动作只在上一个网页完全加载完毕之后才能出现。这也是selenium致密的弱点之一。

In [49]:
browser=Chrome()
browser.get('https://mirrors.tuna.tsinghua.edu.cn/')#打开清华镜像
el=browser.find_element(By.XPATH,'//*[@id="mirror-list"]/table/tbody/tr[188]/td[1]/a[1]')
#找到Ubuntu按钮(find_elements_by_xpath等方法现已废除)
el.click()#点击按钮
#找到我们所想要的内容的位置，并且拉取到本地。这里假设我们想要的是Ubuntu下所有的下载文件名称。
# div_mot=browser.find_element(By.CLASS_NAME,'//*[@id="i_cecream"]/div/main/div/div[1]/div/div/div[3]')
div_list=browser.find_elements(By.XPATH,'//*[@id="list"]/tbody/tr')
a_list=[]
for div in div_list:
    a_list.append(div.find_element(By.XPATH,'./td/a').text)
print(a_list)

['Parent directory/', 'dists/', 'indices/', 'pool/', 'project/', 'ubuntu/', 'ls-lR.gz']


### selenium抓拉勾网


In [60]:
from selenium.webdriver.common.keys import Keys #模拟按键的库
import time
web=Chrome()
web.get('https://www.lagou.com/')
el=web.find_element(By.XPATH,'//*[@id="changeCityBox"]/ul/li[1]/a')
el.click()
time.sleep(1)
web.find_element(By.XPATH,'//*[@id="search_input"]').send_keys('python',Keys.ENTER)
time.sleep(1)
#接下来要做的是，点开其中一个职位的详情界面获取应聘要求。我们要解决的问题是如何进入到新窗口中进行提取。
el=web.find_element(By.XPATH,'//*[@id="openWinPostion"]')
el.click()
time.sleep(1)

注意，在selenium眼中，新窗口是默认不切换过来的。它还停留在原来的窗口中。

In [61]:
#接下来介绍如何进入到新窗口中进行提取。
web.switch_to.window(web.window_handles[-1])#切换到网页选项卡中的最后一个窗口，也就是新打开的窗口
job_detail=web.find_element(By.XPATH,'//*[@id="job_detail"]/dd[2]/div').text
print(job_detail)

岗位职责：
1.负责大模型（包括chatgpt，文心一言等）的promt优化,basecase解决;
2.负责公司AI系统的设计与研发。
3.负责公司AI类模型的工程化落地。
4.负责公司AI服务的在线数据流建设。
任职资格：
1.扎实的计算机基础，出色的编程能力。
2.熟练掌握python，掌握c++，java，go之一。
3.对在线服务的架构设计，性能优化有着不错的理解和实践。
4.熟练使用Mysql/Redis/Kafka/Spark等在线数据存储和加工的常见组件。
5.良好的系统设计能力，对大流量和重计算的分布式系统有着比较丰富的设计经验。
6.熟悉云原生平台，对日志系统，常用的服务器命令和底层原理有着不错的理解。
7.对推荐系统，自然语言处理，机器视觉等方向有一定了解，并且有强烈的兴趣。
8.优秀的技术研究能力，能对新领域展开深入研究。


In [62]:
#接下来介绍怎么关闭页面和退回到原来的页面。
web.close()#关闭当前所在页面
web.switch_to.window(web.window_handles[0])#利用网页的选项卡回退到原来的网页。

接下来介绍一下web的另外一种切换方式。

如果有的时候爬视频遇到了iframe（就是视频框内嵌入了另外一个网页，不能从当前网页直接获取），必须先拿到iframe，然后切换视角到iframe，最后才能够拿到数据。

下面以茶杯狐为例，尝试抓取电影《潜水》的视频数据。

In [66]:
web=Chrome()
web.get('https://www.cqjschungao.com/cupfoxplay/174372-1-1/')
iframe=web.find_element(By.XPATH,'//*[@id="playleft"]/iframe')
#因为一个网页中可能会镶嵌多个iframe,所以需要用特定的方式来指定。
web.switch_to.frame(iframe)
#现在驱动就跳到iframe里面去了
video_tag=web.find_element(By.XPATH,'//*[@id="dplayer"]/div[2]/video')
src=video_tag.get_attribute('src')
print(src)

blob:https://www.gszyv.com/4f17a5fc-0ceb-4a3d-b5d0-f788534d9b17


In [67]:
#下面介绍怎么从iframe中跳出来到原来的网页窗口
web.switch_to.default_content()#默认的窗口就是打开的那个页面。

### 总结
selenium十分的方便，只要是人在网上看到的东西，selenium都能够拿到手，而且代码写起来就像人说话一样，非常的具体和容易；但是其弊端也非常显著：一是非常的慢，必须等网页加载完了才继续；二是容易被反扒识别，一旦被发现几乎无解。